# Imports

In [58]:
import json, os
from graphviz import Digraph

# Special Regex Characters

In [59]:
EPSILON = "ε" #ε
PLUS = '+'
STAR = '*'
Q_MARK = '?'
OR = '|'

ALL_LETTERS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
ALL_NUMBERS = "1234567890"
ALL_SPECIAL = " ~[{]}!@#-_+=$%^&*()?><"

ALL_CHARACTERS = ALL_LETTERS+ALL_NUMBERS+ALL_SPECIAL

# Regex Validation

In [60]:
def firstElementValidation(regex):
    if len(regex) > 0:
        c = regex[0]
        if c == PLUS or c == STAR or c == '/' or c == Q_MARK:
            return False
    return True

# A function that makes sure each opening bracket has its closing end using a stack approach
def regexBracketValidation(regex):
    all_brackets = []
    index = 0
    for c in regex:
        back_iterator = index - 1
        index += 1
        backslash_count = 0
        while back_iterator >= 0 and regex[back_iterator] == '\\':
            backslash_count += 1
            back_iterator -= 1
        if backslash_count % 2 == 1:
            continue
        if c == '(':
            flag = False
            for b in all_brackets:
                if b == '[':
                    flag = True
                    break
            if flag:
                continue
            all_brackets.append(c)
        elif c == ')':
            flag = False
            for b in all_brackets:
                if b == '[':
                    flag = True
                    break
            if flag:
                continue
            if len(all_brackets) > 0:
                if all_brackets[-1] == '(':
                    all_brackets.pop()
                else:
                    return False
            else:
                return False
        elif c == '[':
            flag = False
            for b in all_brackets:
                if b == '[':
                    flag = True
                    break
            if flag:
                continue
            all_brackets.append(c)
        elif c == ']':
            if len(all_brackets) > 0:
                if all_brackets[-1] == '[':
                    all_brackets.pop()
                else:
                    return False
        elif c == '{':
            all_brackets.append(c)
        elif c == '}':
            if len(all_brackets) > 0:
                if all_brackets[-1] == '{':
                    all_brackets.pop()
                else:
                    return False
            else:
                return False
        elif c == Q_MARK or c == STAR or c == PLUS:
            if (len(all_brackets) > 0 and all_brackets[-1] == '[') or index == 0:
                continue
            elif regex[index - 2] == Q_MARK or regex[index - 2] == STAR or regex[index - 2] == PLUS:
                return False

    if len(all_brackets) > 0:
        return False
    return True

# A function that makes sure that each backslash (not having a first backslash) has a following character
def lastBackslashValidation(regex):
    if regex[-1] == '\\':
        back_iterator = len(regex) - 2
        backslash_count = 0
        while back_iterator >= 0 and regex[back_iterator] == '\\':
            backslash_count += 1
            back_iterator -= 1
        return backslash_count % 2 == 1
    return True

# A function that makes sure the contents of square brackets in a given regex are valid
def squareBracketContentsValidation(regex):
    if len(regex) == 0 or (len(regex) == 1 and regex[0] == '^'):
        return False
    index = 0
    dash_indices = []
    for c in regex:
        if index == 1 and c == '-' and regex[0] == '^':
            index += 1
            continue
        if c == '-' and index > 0 and index < len(regex) - 1:
            if len(dash_indices) > 0:
                old_index = dash_indices[-1]
                if index == old_index + 1 or index == old_index + 2:
                    index += 1
                    continue
            dash_indices.append(index)
        index += 1
    for dash_index in dash_indices:
        if regex[dash_index - 1] > regex[dash_index + 1]:
            return False
    return True

# A function that makes sure the contents of brackets are valid as if the contents of brackets are a regex on their own
def bracketContentsValidation(regex):
    index = 0
    bracket_indices_stack = []

    for c in regex:
        if c == '(':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if found_opening_bracket:
                continue
            bracket_indices_stack.append(index)
        elif c == ')':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if found_opening_bracket:
                continue
            opening_index = bracket_indices_stack.pop()
            if not firstElementValidation(regex[opening_index+1:index]):
                return False
        elif c == '[':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if found_opening_bracket:
                continue
            bracket_indices_stack.append(index)
        elif c == ']':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if not found_opening_bracket:
                continue
            opening_index = bracket_indices_stack.pop()
            if not squareBracketContentsValidation(regex[opening_index+1:index]):
                return False
        index += 1
    return True

# A function using all the previous functions to validate a regex
def validateRegex(regex):
    if len(regex) == 0:
        return True
    if not firstElementValidation(regex):
        return False
    if not regexBracketValidation(regex):
        return False
    if not lastBackslashValidation(regex):
        return False
    if not bracketContentsValidation(regex):
        return False
    return True


# NFA Creator

Pre-processing:

In [77]:
# A function used to programmatically add extra brackets that helps in ORing (alternation) of operands for the OR '|' operator
def addOrBrackets(regex):
    or_indices = []
    brackets_stack = []
    index = -1
    for c in regex:
        index += 1
        if (c == '(' or c == '[') and (len(brackets_stack) == 0 or brackets_stack[-1] != '['):
            brackets_stack.append(c)
        elif c == ']' and (len(brackets_stack) == 0 or brackets_stack[-1] == '['):
            brackets_stack.pop()
        elif c == ')' and (len(brackets_stack) == 0 or brackets_stack[-1] != '['):
            brackets_stack.pop()
        elif c == '|' and (len(brackets_stack) == 0 or brackets_stack[-1] != '['):
            or_indices.append(index)

    if len(or_indices) == 0:
        return regex
    regex_parts = []
    index = -1
    iterator = 0
    regex_part = ''
    for c in regex:
        index += 1
        if iterator < len(or_indices) and index == or_indices[iterator]:
            regex_parts.append(regex_part)
            regex_part = ''
            iterator += 1
        else:
            regex_part += c
    regex_parts.append(regex_part)
    print(regex_parts)

    index = -1
    for regex_part in regex_parts:
        index += 1
        opening_brackets_count = 0
        sub_index = -1
        marker = None
        first_open_bracket_index = None
        first_close_bracket_index = None
        for c in regex_part:
            sub_index += 1
            if c == '(':
                if first_open_bracket_index is None:
                    first_open_bracket_index = sub_index
                opening_brackets_count += 1
                marker = sub_index
            elif c == ')':
                if first_close_bracket_index is None:
                    first_close_bracket_index = sub_index
                opening_brackets_count -= 1
                marker = sub_index
        if marker is None or (opening_brackets_count == 0 and (first_open_bracket_index < first_close_bracket_index)):
            regex_part = '(' + regex_part + ')'
        else:
            sub_index = -1
            new_regex_part = ''
            for c in regex_part:
                sub_index += 1                    
                if opening_brackets_count > 0:
                    if sub_index == first_open_bracket_index:
                        new_regex_part += '('
                elif opening_brackets_count < 0:
                    if sub_index == first_close_bracket_index:
                        new_regex_part += ')'
                else:
                    if sub_index == first_open_bracket_index:
                        new_regex_part += '('
                    elif sub_index == first_close_bracket_index:
                        new_regex_part += ')'

                new_regex_part += c
            if opening_brackets_count > 0:
                new_regex_part += ')'
            elif opening_brackets_count < 0:
                new_regex_part = '(' + new_regex_part
            regex_part = new_regex_part
        regex_parts[index] = regex_part
    print(regex_parts)

    new_regex = regex_parts[0]
    index = -1
    for _ in regex_parts:
        index += 1
        if index == len(regex_parts) - 1:
            break
        new_regex += '|' + regex_parts[index + 1]
    return new_regex


Lexing:

In [62]:
# A function that takes contents of a square brackets, and parse it using regex rules returning
# the allowed or not allowed (in case of negation) characters
def lexSquareBrackets(regex, level):
    is_allowed = True
    allowed_characters = []
    index = 0
    dash_indices = []
    for c in regex:
        if index == 1 and c == '-' and regex[0] == '^':
            index += 1
            continue
        if c == '-' and index > 0 and index < len(regex) - 1:
            if len(dash_indices) > 0:
                old_index = dash_indices[-1]
                if index == old_index + 1 or index == old_index + 2:
                    index += 1
                    continue
            dash_indices.append(index)
        index += 1
    if regex[0] == '^':
        is_allowed = False

    allowed_tuples = []
    for dash_index in dash_indices:
        current_tuple = (regex[dash_index - 1], regex[dash_index + 1])
        if current_tuple not in allowed_tuples:
            allowed_tuples.append(current_tuple)

    allowed_all = []

    index = -1
    dash_index = 0
    for c in regex:
        index += 1
        if index == 0 and regex[0] == '^':
            continue
        if dash_index < len(dash_indices):
            if index == dash_indices[dash_index] or index == dash_indices[dash_index] - 1:
                continue
            elif index == dash_indices[dash_index] + 1:
                dash_index += 1
                continue
        add_c = True
        for t in allowed_tuples:
            if c >= t[0] and c <= t[1]:
                add_c = False
        if add_c and c not in allowed_characters:
            allowed_characters.append(c)

    for c in allowed_characters:
        allowed_all.append(c)
    for t in allowed_tuples:
        allowed_all.append(t)
    
    # print(allowed_all)
    
    return (allowed_all, level, is_allowed)

# The main recursive function used to parse the regex returning a list of tuples (character, level, extra)
# The list of tuples is then used to create the NFA states
def lexBrackets(regex, level = 0):
    i = -1
    last_ignorer = 0
    indices = []
    for c in regex:
        i += 1
        if i < last_ignorer:
            continue
        if c == '[':
            indices.append(i)
            character_level_extra.append((c, level, None))
            brackets_indices = [i]
            for j in range(i + 1, len(regex)):
                if regex[j] == ']':
                    brackets_indices.pop()
                    if len(brackets_indices) == 0:
                        character_level_extra.append(lexSquareBrackets(regex[i+1:j], level + 1))
                        last_ignorer = j
                        break
        elif c == '(':
            indices.append(i)
            character_level_extra.append((c, level, None))
            brackets_indices = [i]
            for j in range(i + 1, len(regex)):
                if regex[j] == '(':
                    brackets_indices.append(j)
                elif regex[j] == ')':
                    brackets_indices.pop()
                    if len(brackets_indices) == 0:
                        print(regex[i+1:j])
                        lexBrackets(regex[i+1:j], level + 1)
                        last_ignorer = j
                        break
        elif c == Q_MARK or c == STAR or c == PLUS or c == OR:
            if c == OR:
                if i > 0:
                    previous = regex[i - 1]
                    if previous == Q_MARK or previous == STAR or previous == PLUS:
                        character_level_extra.append((c, level, None))
                else:
                    character_level_extra.append((EPSILON, level, OR))
            continue
        else:
            if c == '\\':
                last_ignorer += 2
            if not(c == ')' or c == ']'):
                # print(regex[i])
                pass
            extra = None
            if i + 1 < len(regex):
                next = regex[i + 1]
                if next == Q_MARK or next == STAR or next == PLUS or next == OR:
                    extra = next
                elif regex[i] == '\\':
                    extra = next
                    if i + 2 < len(regex):
                        next_of_next = regex[i + 2]
                        if next_of_next == Q_MARK or next_of_next == STAR or next_of_next == PLUS or next_of_next == OR:
                            extra += next_of_next
            character_level_extra.append((c, level, extra))
            indices.append(i)
    characters = []
    for j in indices:
        characters.append(regex[j])
    # print(regex, characters)
    # print(regex, indices)
    return character_level_extra


Post-processing:

In [63]:
# This function is used to remove unnecessary brackets within the output of the parsed regex
def removeUnnecessaryBrackets(character_level_extra):
    while True:
        found = False
        index = -1
        for c_l_e in character_level_extra:
            index += 1
            character = c_l_e[0]
            level = c_l_e[1]
            if character == '(':
                for sub_index in range(index, len(character_level_extra)):
                    if character_level_extra[sub_index][0] == ')' and character_level_extra[sub_index][1] == level:
                        if character_level_extra[sub_index][2] == None:
                            if index > 0 and character_level_extra[index - 1][2] != OR and character_level_extra[index - 1][0] != OR:
                                # print('Removing:', c_l_e, character_level_extra[sub_index], index, sub_index)
                                found = True
                                character_level_extra.pop(sub_index)
                                character_level_extra.pop(index)
                        break

        if not found:
            break
    return character_level_extra

# This function is used to fill necessary yet empty brackets with EPSILONS
def fillEmptyBrackets(character_level_extra):
    index = -1
    for c_l_e in character_level_extra:
        index += 1
        character = c_l_e[0]
        level = c_l_e[1]
        if character == '(':
            if index + 1 < len(character_level_extra) and character_level_extra[index + 1][0] == ')' and level == character_level_extra[index + 1][1]:
                character_level_extra.insert(index + 1, (EPSILON, level + 1, None))
    return character_level_extra

# This function is used to fill the surroundings of OR operators in case of being empty with EPSILONS
# This function is probably redundant after adding the pre-processing block
def fillEmptyOrsRight(character_level_extra):
    index = -1
    for c_l_e in character_level_extra:
        index += 1
        level = c_l_e[1]
        extra = c_l_e[2]
        if extra is not None and type(extra) is not bool:
            if len(extra) == 1:
                if extra == OR:
                    if index == len(character_level_extra) - 1 or character_level_extra[index + 1][0] == ')':
                        character_level_extra.insert(index + 1, (EPSILON, level, None))
            elif extra[-1] == OR:
                if index == len(character_level_extra) - 1 or character_level_extra[index + 1][0] == ')':
                        character_level_extra.insert(index + 1, (EPSILON, level, None))
    return character_level_extra


States creation:

In [64]:
global states
states = []

# This recursive function parses the list of tuples of character, level and extra to create the NFA states
def createStates(character_level_extra, state_index = 1, previous_state_index = 0, next_state_index = None):
    base_call = False
    if next_state_index is None:
        base_call = True
        states.clear()
        states.append(('S0', False, []))
    else:
        absolute_next_index = next_state_index
        absolute_previous_index = previous_state_index

    skipper = 0
    state_indexer = -1
    or_flag = None
    for c_l_e in character_level_extra:
        state_indexer += 1
        if skipper > 0:
            skipper -= 1
            continue

        # in-state
        previous_state = states[previous_state_index]

        character, level, extra = c_l_e

        # Create the new state to be next (out-state)
        if next_state_index is None or previous_state_index == next_state_index or (not base_call and (or_flag is not None or state_indexer == 0)):
            next_state_index = state_index
            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            next_state = states[state_index]

            state_index += 1
        else:
            next_state = states[next_state_index]


        if character == '[':
            skipper = 1
            available_index = -1
            characters = ''
            for c_t in character_level_extra[state_indexer + 1][0]:
                available_index += 1
                add_comma = False
                if available_index < len(character_level_extra[state_indexer + 1][0]) - 1:
                    add_comma = True
                if type(c_t) is str:        # Character
                    characters += c_t
                elif type(c_t) is tuple:    # Tuple
                    characters += (c_t[0] + '-' + c_t[1])
                if add_comma:
                    characters += ', '
            # Create the new states
            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state1 = states[state_index]
            state_index += 1

            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state2 = states[state_index]
            state_index += 1

            if character_level_extra[state_indexer + 1][2] == False:
                characters = "NOT " + characters

            previous_state[2].append((EPSILON, current_state1[0]))
            current_state1[2].append((characters, current_state2[0]))
            current_state2[2].append((EPSILON, next_state[0]))

        elif character == '(':
            for iterator in range(state_indexer, len(character_level_extra)):
                current_character = character_level_extra[iterator][0]
                current_level = character_level_extra[iterator][1]
                current_extra = character_level_extra[iterator][2]
                if current_character == ')' and level == current_level:
                    skipper = iterator - state_indexer
                    # print('Call', previous_state_index, next_state_index)
                    c1_index, c2_index, state_index = createStates(character_level_extra[state_indexer + 1:iterator], state_index, previous_state_index, next_state_index)
                    # print('Return', c1_index, c2_index)
                    current_state1 = states[c1_index]
                    current_state2 = states[c2_index]
                    # print(previous_state, current_state1, current_state2, next_state)
                    extra = current_extra
                    if extra is not None and iterator + 1 < len(character_level_extra) and character_level_extra[iterator + 1][0] == OR:
                        current_extra = OR
                        skipper += 1
                    break
                    
        elif character == OR or character == ']' or character == ')':
            pass
        else:
            if character == '\\':
                skipper = 1
                if len(extra) == 1:
                    character += extra
                    extra = None
                else:
                    character += extra[0]
                    extra = extra[1]
            if character == '.':
                character = "CHAR"
                character = '.'
            elif character == '\\w':
                character = "ALPHANUM"
            elif character == '\\W':
                character = "SPECIAL"
            elif character == '\\d':
                character = "DIGIT"
            elif character == '\\D':
                character = "NON-DIGIT"
            # Create the new current states
            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state1 = states[state_index]

            state_index += 1

            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state2 = states[state_index]

            state_index += 1

            previous_state[2].append((EPSILON, current_state1[0]))
            current_state1[2].append((character, current_state2[0]))
            current_state2[2].append((EPSILON, next_state[0]))
            
        if extra == STAR:
            current_state2[2].append((EPSILON, current_state1[0]))
            current_state1[2].append((EPSILON, next_state[0]))
        elif extra == PLUS:
            current_state2[2].append((EPSILON, current_state1[0]))
        elif extra == Q_MARK:
            current_state1[2].append((EPSILON, next_state[0]))
        
        if character == '(':
            extra = current_extra
            # print('Here extra character', character, current_extra)

        if extra != OR and character != OR and character != '[' and state_indexer < len(character_level_extra) - 1 and character_level_extra[state_indexer + 1][0] != OR:
            # print('Proceeding Here', c_l_e, character_level_extra[state_indexer + 1], previous_state_index, next_state_index)
            previous_state_index = next_state_index
            or_flag = True
        else:
            or_flag = None

    if base_call and state_indexer == len(character_level_extra) - 1:
        if next_state_index is None:
            next_state_index = 0
        t_state = states[next_state_index]
        states[next_state_index] = (t_state[0], True, t_state[2])
        print('Terminating State:', states[next_state_index],'\n')

    if not base_call:
        next_state[2].append((EPSILON, 'S'+str(absolute_next_index)))
        return absolute_previous_index, next_state_index, state_index
    else:
        for state in states:
            state = (state[0], state[1], list(dict.fromkeys(state[2])))
    

NFA output:

In [65]:
# This function writes the NFA states in a json file
def writeNfa(states):
    open('NFA.json', 'w').close()

    with open('NFA.json', 'w', encoding='utf-8') as f:
        json_string = ""
        dict_to_write = {"startingState": "S0"}
        for state in states:
            sub_dict = {}
            sub_dict["isTerminatingState"] = state[1]
            for input_output in state[2]:
                if input_output[0] not in sub_dict:
                    sub_dict[input_output[0]] = []
                sub_dict[input_output[0]].append(input_output[1])
            dict_to_write[state[0]] = sub_dict
        json_string = json.dumps(dict_to_write, indent=3)
        f.write(json_string)

# This function outputs the NFA states to a png file illustrating the states and their transitions
def drawNfa(view_graph):
    nfa_graph = Digraph(graph_attr={'rankdir': 'LR'})
    with open('NFA.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

    for state in data:
        if state == "startingState":
            continue
        if data[state]["isTerminatingState"] == False:
            nfa_graph.attr("node", shape = 'circle')
        else:
            nfa_graph.attr("node", shape = 'doublecircle')
        nfa_graph.node(state)
        if state == data["startingState"]:
            nfa_graph.attr("node", shape = 'none')
            nfa_graph.node('')
            nfa_graph.edge("", state)
    for state in data:
        if state == "startingState":
            continue
        for edges in data[state]:
            if edges == "isTerminatingState":
                continue
            for edge in data[state][edges]:
                nfa_graph.edge(state, edge, edges)

    picFile = "NFA"
    nfa_graph.render(picFile, view = view_graph, format = 'png', overwrite_source = True)
    os.remove("NFA")

NFA flow:

In [66]:
# This function holds the flow from the input regex to the output NFA
def nfaFlow(view_graph = False):
    input_regex = input("\nEnter regular expression: ")
    if validateRegex(input_regex):
        print('\nValid regex:', input_regex, '\n')
        input_regex = addOrBrackets(input_regex)
        global character_level_extra
        character_level_extra = []
        character_level_extra = lexBrackets(input_regex)

        character_level_extra = fillEmptyOrsRight(character_level_extra)
        character_level_extra = removeUnnecessaryBrackets(character_level_extra)
        character_level_extra = fillEmptyBrackets(character_level_extra)
        print('Characters to parse:', character_level_extra, '\n')

        createStates(character_level_extra)
        print('NFA states: ',states, '\n')

        writeNfa(states)
        drawNfa(view_graph)

        print('Input regex: ', input_regex + '\n')
        return True   
    else:
        print('Invalid regex')
        return False

# DFA Creator

Reading NFA:

In [67]:
# This function reads the NFA json file and restore its different states with their transitions
def getNfaStates():
    with open('NFA.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

    states = {}
    for state in data:
        if state == "startingState":
            continue
        input_next = []
        for edges in data[state]:
            if edges == "isTerminatingState":
                continue
            for edge in data[state][edges]:
                input_next.append((edges, edge))
        states[state] = (data[state]["isTerminatingState"], input_next)

    return states


Epsilon-closure:

In [68]:
# This function removes EPSILONS and combines states that are connected together using EPSILONS
def epsilonlessNfaStates(nfa_states):
    possible_epsiloness_states = {}

    for state in nfa_states:
        current_state_to_be = [state]
        index = -1
        while index < len(nfa_states) - 1:
            index += 1
            for current_state in current_state_to_be:
                for input_next in nfa_states[current_state][1]:
                    if input_next[0] == EPSILON and input_next[1] not in current_state_to_be:
                        current_state_to_be.append(input_next[1])

        current_state_to_be.sort()
        possible_epsiloness_states[state] = current_state_to_be

    start_states = ['S0']
    previous_input = {'S0': None}
    for state in nfa_states:
        for input_next in nfa_states[state][1]:
            if input_next[0] != EPSILON and input_next[1] not in start_states:
                start_states.append(input_next[1])
                previous_input[input_next[1]] = (state, input_next[0])

    to_be_deleted = []
    epsilonless_states = {}
    for state in possible_epsiloness_states:
        if state not in start_states:
            to_be_deleted.append(state)
        else:
            epsilonless_states[state] = (False, [])
    for state in to_be_deleted:
        del possible_epsiloness_states[state]
    
    for state in previous_input:
        is_terminating = False
        for sub_state in possible_epsiloness_states[state]:
            if nfa_states[sub_state][0] == True:
                is_terminating = True
                break
        epsilonless_states[state] = (is_terminating, epsilonless_states[state][1])
        if previous_input[state] is None:
            continue
        for mother_state in epsilonless_states:
            if previous_input[state][0] in possible_epsiloness_states[mother_state]:
                # print('Adding', (previous_input[state][1], state), 'to', mother_state)
                epsilonless_states[mother_state][1].append((previous_input[state][1], state))

    return epsilonless_states


DFA states creation:

In [69]:
# This function takes the epsilonless states and converts to DFA states
def createDfaStates(nfa_states):
    is_already_dfa = True
    for state in nfa_states:
        possible_inputs = []
        for input_next in nfa_states[state][1]:
            if input_next[0] in possible_inputs:
                is_already_dfa = False
                break
            possible_inputs.append(input_next[0])
        if not is_already_dfa:
            break
    if is_already_dfa:
        print('\nNFA is already a DFA\n')
        return nfa_states
    # print('\nNFA states:', nfa_states, '\n')

    possible_inputs = set()
    for state in nfa_states:
        print(state, nfa_states[state][1])
        for input_next in nfa_states[state][1]:
            possible_inputs.add(input_next[0])
    # print('Possible inputs:', possible_inputs, '\n')

    explored_sets_of_states = []

    dfa_states = {'S0': (nfa_states['S0'][0], [])}

    print('Possible inputs:', possible_inputs)
    for possible_input in possible_inputs:
        print(possible_input)
        current_state_to_be = ['S0']
        index = -1
        while index < len(nfa_states) - 1:
            index += 1
            for current_state in current_state_to_be:
                for input_next in nfa_states[current_state][1]:
                    if input_next[0] == possible_input and input_next[1] not in current_state_to_be:
                        current_state_to_be.append(input_next[1])
        current_state_to_be.sort()
        current_state_to_be.remove('S0')
        if current_state_to_be not in explored_sets_of_states:
            explored_sets_of_states.append(current_state_to_be)
        if len(current_state_to_be) > 0:
            dfa_states['S0'][1].append((possible_input, current_state_to_be))

    print('Initial dfa states:', dfa_states)

    set_of_new_states = {}

    explored_state_index = 0
    for states in explored_sets_of_states:
        explored_state_index += 1
        set_of_new_states['S'+str(explored_state_index)] = states

    for state in nfa_states:
        for input_next in nfa_states[state][1]:
            possible_inputs.add(input_next[0])

    state_index = 0
    for set_of_states in explored_sets_of_states:
        state_index += 1
        is_terminating = False
        for state in set_of_states:
            if nfa_states[state][0] == True:
                is_terminating = True
        
        list_of_transitions = []
        for possible_input in possible_inputs:
            current_state_to_be = []
            states_to_loop_on = []
            states_to_loop_on.extend(set_of_states)

            for current_state in states_to_loop_on:
                for input_next in nfa_states[current_state][1]:
                    if input_next[0] == possible_input and input_next[1] not in current_state_to_be:
                        current_state_to_be.append(input_next[1])

            current_state_to_be.sort()
            if current_state_to_be not in explored_sets_of_states:
                explored_state_index += 1
                explored_sets_of_states.append(current_state_to_be)
                set_of_new_states['S'+str(explored_state_index)] = current_state_to_be
            if len(current_state_to_be) > 0:
                for state_name in set_of_new_states:
                    if set_of_new_states[state_name] == current_state_to_be:
                        list_of_transitions.append((possible_input, state_name))
                        break
        if len(list_of_transitions) > 0 or is_terminating:
            dfa_states['S'+str(state_index)] = (is_terminating, list_of_transitions)

    # Renaming states of start state:
    new_names = []
    for input_next in dfa_states['S0'][1]:
        for state_name in set_of_new_states:
            if set_of_new_states[state_name] == input_next[1]:
                new_names.append((input_next[0], state_name))
                break
    dfa_states['S0'] = (dfa_states['S0'][0], new_names)

    return dfa_states


DFA output:

In [70]:
# This function outputs the DFA states to a png file illustrating the states and their transitions
def drawDfa(dfa_states, view_graph = False, another_name = 'DFA'):
    dfa_graph = Digraph(graph_attr={'rankdir': 'LR'})

    index = 0
    for state in dfa_states:
        if dfa_states[state][0] == False:
            dfa_graph.attr("node", shape = 'circle')
        else:
            dfa_graph.attr("node", shape = 'doublecircle')
        dfa_graph.node(state)
        if index == 0:
            index += 1
            dfa_graph.attr("node", shape = 'none')
            dfa_graph.node('')
            dfa_graph.edge("", state)
    for state in dfa_states:
        for input_next in dfa_states[state][1]:
            dfa_graph.edge(state, input_next[1], input_next[0])

    picFile = another_name
    dfa_graph.render(picFile, view = view_graph, format = 'png', overwrite_source = True)
    os.remove(another_name)


DFA flow:

In [71]:
# This function holds the flow from the input NFA to the output DFA
def dfaFlow(view_graph = False):
    nfa_states = getNfaStates()
    epsilonless_nfa_states = epsilonlessNfaStates(nfa_states)
    print('Epsilonless states:', epsilonless_nfa_states)
    drawDfa(epsilonless_nfa_states, another_name = 'Epsilonless_NFA')
    dfa_states = createDfaStates(epsilonless_nfa_states)
    print('\nDfa states:', dfa_states)
    drawDfa(dfa_states, view_graph)
    return dfa_states

# Dfa Minimizer

DFA minimizer:

In [72]:
# This function reduces the DFA to a minimized DFA by combining states that have the same behaviour
def minimizeDfa(dfa_states):
    possible_inputs = []
    for state in dfa_states:
        for input_next in dfa_states[state][1]:
            if input_next[0] not in possible_inputs:
                possible_inputs.append(input_next[0])

    accepting_states = []
    non_accepting_states = []


    for state in dfa_states:
        if dfa_states[state][0] == True:
            accepting_states.append(state)
        else:
            non_accepting_states.append(state)

    groups_to_process = []
    if len(accepting_states) > 0:
        groups_to_process.append(accepting_states)
    if len(non_accepting_states) > 0:
        groups_to_process.append(non_accepting_states)

    print('\nInitially non-accepting states:', accepting_states)
    print('\nInitially accepting states:', non_accepting_states)

    while True:
        breaker = True

        copy_to_process = groups_to_process.copy()

        index = -1
        for group in copy_to_process:
            index += 1
            if len(group) > 1:
                out_liers = []
                out_lier_behavior = None
                for possible_input in possible_inputs:
                    elements_behavior = []
                    for element in group:
                        next_state = None
                        for input_next in dfa_states[element][1]:
                            if input_next[0] == possible_input:
                                next_state = input_next[1]
                                break
                        if next_state is not None:
                            sub_index = -1
                            for g in copy_to_process:
                                sub_index += 1
                                if next_state in g:
                                    elements_behavior.append(sub_index)
                                    break
                        else:
                            elements_behavior.append(-1)
                    # print('\nElements behaviour:', elements_behavior, 'for input', possible_input)

                    frequency_dict = {}
                    for behavior in elements_behavior:
                        if behavior not in frequency_dict:
                            frequency_dict[behavior] = 1
                        else:
                            frequency_dict[behavior] = frequency_dict[behavior] + 1

                    minimum_behavior = len(dfa_states)
                    if len(frequency_dict) != 1:
                        for behavior in elements_behavior:
                            if frequency_dict[behavior] < minimum_behavior:
                                minimum_behavior = frequency_dict[behavior]
                    
                        for behavior in elements_behavior:
                            if frequency_dict[behavior] == minimum_behavior:
                                out_lier_behavior = behavior
                                break
                    if out_lier_behavior is not None:
                        sub_index = -1
                        for behavior in elements_behavior:
                            sub_index += 1
                            if behavior == out_lier_behavior:
                                out_liers.append(group[sub_index])
                        # print('\nElements:', out_liers, 'does not belong to group', group)
                        breaker = False
                        break
            if not breaker:
                for out_lier in out_liers:
                    group.remove(out_lier)
                copy_to_process[index] = group
                copy_to_process.append(out_liers)
                # print('\nNew groups:', copy_to_process)
                break

        if breaker:
            break

        groups_to_process = copy_to_process
        
    print('\nFinal groups:', groups_to_process)

    for group in groups_to_process:
        if len(group) > 1:
            if 'S0' in group:
                representative = 'S0'
            else:
                representative = group[0]
            for state in dfa_states:
                index = -1
                for input_next in dfa_states[state][1]:
                    index += 1
                    if input_next[1] in group:
                        dfa_states[state][1][index] = (dfa_states[state][1][index][0], representative)
            for element in group:
                if element == representative:
                    continue
                del dfa_states[element]
    print('\nMinimized Dfa:', dfa_states)
    return dfa_states


DFA minimizer output:

In [73]:
# This function outputs the minimized DFA states to a json file
# as well as to a png file illustrating the states and their transitions
def writeMinimizedDfa(dfa_states):
    with open('Minimized_DFA.json', 'w', encoding='utf-8') as f:
        dict_to_write = {"startingState": "S0"}
        for state in dfa_states:
            sub_dict = {}
            sub_dict["isTerminatingState"] = dfa_states[state][0]
            for input_next in dfa_states[state][1]:
                sub_dict[input_next[0]] = input_next[1]
            dict_to_write[state] = sub_dict
        json_string = json.dumps(dict_to_write, indent=3)
        f.write(json_string)

DFA flow:

In [74]:
# This function holds the flow from the input DFA to the output minimized DFA
def minimizedDfaFlow(dfa_states):
    minimized_dfa = minimizeDfa(dfa_states)
    drawDfa(minimized_dfa, another_name='Minimized_DFA')
    writeMinimizedDfa(minimized_dfa)        

# Main Flow

In [75]:
# This function holds the whole flow from the input regex to the output minimized DFA
def allFlow():
    if nfaFlow() == True:
        minimizedDfaFlow(dfaFlow())

allFlow()


Valid regex: a*|b 

['a*', 'b']
['(a*)', '(b)']
a*
b
Characters to parse: [('(', 0, None), ('a', 1, '*'), (')', 0, '|'), ('(', 0, None), ('b', 1, None), (')', 0, None)] 

Terminating State: ('S1', True, []) 

NFA states:  [('S0', False, [('ε', 'S3'), ('ε', 'S6')]), ('S1', True, []), ('S2', False, [('ε', 'S1')]), ('S3', False, [('a', 'S4'), ('ε', 'S2')]), ('S4', False, [('ε', 'S2'), ('ε', 'S3')]), ('S5', False, [('ε', 'S1')]), ('S6', False, [('b', 'S7')]), ('S7', False, [('ε', 'S5')])] 

Input regex:  (a*)|(b)

Epsilonless states: {'S0': (True, [('a', 'S4'), ('b', 'S7')]), 'S4': (True, [('a', 'S4')]), 'S7': (True, [])}

NFA is already a DFA


Dfa states: {'S0': (True, [('a', 'S4'), ('b', 'S7')]), 'S4': (True, [('a', 'S4')]), 'S7': (True, [])}

Initially non-accepting states: ['S0', 'S4', 'S7']

Initially accepting states: []

Final groups: [['S4'], ['S7'], ['S0']]

Minimized Dfa: {'S0': (True, [('a', 'S4'), ('b', 'S7')]), 'S4': (True, [('a', 'S4')]), 'S7': (True, [])}
